# Ifsoac y sus transformaciones

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

class Ifsoac:
    def __init__(self, serie=None, nlados=500, p=1/2, tam_punto=.01, ventana=9, 
                 grafica=True, transform=None, scale=.8):
        if serie is None:
            r = np.random.rand(1000000)
        else:
            r = serie
        # nlados depende del tamaño del rango de la serie
        """
        Ej. Si len(serie)==100000 pero cada elemento de la serie tiene
        dos dígitos en la parte entera y dos en la parte decimal, ¿de que 
        tamaño será el rango? (cuantos valores diferentes posibles hay)
        (10*10)*(10*10)= 10000 es el tamaño del rango. nlados=10000 en este
        caso, para tener un vértice para cada valor de mi serie.
        Si tomo nlados menor que este valor, "perderé" información.
        ej, si la serie toma valores 0,1,2,3,4,5,6,7,8,9 y nlados =3
        0, 3/9=1/3=0, 2/3=0, 3/(3+e)=0, 4/3=1, 5/3=1, 6/(3+e)=1, 7/3=2, 8/3=2, 9/(3+e)=2
        0,0,0,0,1,1,1,2,2,2
        """
        self.p = p
        self.nlados = nlados
        self.ventana=ventana
        self.tam_punto = tam_punto
        self.scale=scale
        poligono = self._regular(nlados)
        r_normalizada = self.normaliza(r)
        # le sumamos 1e-10 para que no alcance a valer 1
        indices = (r_normalizada * nlados).astype(int)
        self.serie = poligono[indices]
        if grafica:
            if transform is None:
                self.grafica()
            elif transform == 'estereo':
                self.estereo_grafica()
            elif transform == 'escher':
                self.escher_grafica()
            elif transform == 'antipolar':
                self.antipolar_grafica()

    def normaliza(self, serie):
        min, max = np.min(serie), np.max(serie)
        return (serie-min)/(max-min+1e-10)  # r está en [0, 1) 

    def _regular(self, n):
        angulo = np.linspace(np.pi/2, 5*np.pi/2, n, endpoint=False)
        x = np.cos(angulo)
        y = np.sin(angulo)
        return np.array([x,y]).T

    def jdc(self):
        k1, k2 = 0, 0
        def promedio3(a, b):
            nonlocal k1, k2
            k1, k2 = (a + k1)*self.p, (b + k2)*self.p
            return k1, k2
        _jdc = np.frompyfunc(promedio3, 2, 1)
        t = self.serie
        sierpinsky = np.vstack(_jdc(*t.T).astype(np.ndarray))
        return sierpinsky

    def grafica(self):
        plt.figure(figsize=(self.ventana, self.ventana))
        plt.xlim(-1,1)
        plt.ylim(-1,1)
        plt.scatter(*self.jdc().T, marker='.', s=self.tam_punto)
        plt.show()

    def estereo_grafica(self):
        plt.figure(figsize=(self.ventana, self.ventana))
        #plt.xlim(-10,10)
        #plt.ylim(-10,10)
        plt.scatter(*estereografica(self.jdc().T), marker='.', s=self.tam_punto)
        plt.show()

    def escher_grafica(self):
        plt.figure(figsize=(self.ventana, self.ventana))
        #plt.xlim(-10,10)
        #plt.ylim(-10,10)
        plt.scatter(*escher(self.jdc().T), marker='.', s=self.tam_punto)
        plt.show()

    def antipolar_grafica(self):
        plt.figure(figsize=(self.ventana*2*np.pi, self.ventana))
        #plt.xlim(-10,10)
        plt.ylim(self.scale,1)
        plt.scatter(*antipolar(self.jdc().T), marker='o', s=self.tam_punto)
        plt.show()

    def __repr__(self):
        return f"Ifsoac(nlados={self.nlados}, p={self.p})"

def norma(p):
    x, y = p
    return np.sqrt(x * x + y * y)

def estereografica(p):
    return p/norma(p)**2

def escher(p):
    return p/norma(p/norma(p)-p)

def antipolar(p):
    x,y=p
    alfa = np.arctan2(y,x)
    r = norma(p)
    return np.array((alfa, r))

In [ ]:
lorentz = pd.read_csv("/content/drive/MyDrive/Proyectos/INVESTIGACIÓN Y DESARROLLO/Capitulo Springer/lorenz.dat", header=None)
rossler = pd.read_csv("/content/drive/MyDrive/Proyectos/INVESTIGACIÓN Y DESARROLLO/Capitulo Springer/rossler.dat", header=None)

In [ ]:
# Sistemas de funciones iteradas en un círculo
Ifsoac(lorentz[0], tam_punto=1)
Ifsoac(rossler[0], tam_punto=1)

In [ ]:
Ifsoac(lorentz[0], transform="antipolar", tam_punto=10)

In [ ]:
Ifsoac(rossler[0], transform="antipolar", tam_punto=10)

In [ ]:
def logistica(x):
    return 4*x*(1-x)

def iterar(funcion=logistica, x0=0.3, N=100000, N0=100000):
    ll = []
    for i in range(N0):
        y = funcion(x0)
        x0 = y
        #llogistica.append(y)
    # ya se generó una x0 "caótica"
    for i in range(N):
        y = funcion(x0)
        x0 = y
        ll.append(y)
    return ll

def tent(x):
    _lambda = 0.999
    return 2 * x * _lambda if x < 0.5 else (2 - 2 * x) * _lambda

In [ ]:
logi = iterar(logistica)
tienda = iterar(tent, N0=0, x0=.4)

In [ ]:
Ifsoac(logi)
Ifsoac(tienda)

In [ ]:
Ifsoac(logi, transform="antipolar", tam_punto=10, scale=0)
Ifsoac(tienda, transform="antipolar", tam_punto=10, scale=0)

In [ ]:
Ifsoac(logi, transform="estereo", tam_punto=.5)

In [ ]:
tienda = iterar(tent, N0=0, x0=.4)
Ifsoac(tienda, transform="estereo", tam_punto=.5)

# IFSoac con igual cantidad de puntos en cada bin


In [ ]:
lorentz

In [ ]:
def same_amount_bins(df, nbins=500):
    if type(df) == list:
        df = pd.DataFrame(df)
    lo=df.sort_values(0)  # ordenamos por valores
    times = len(lo)/nbins # veces que se repetirá etiqueta de bin
    lo["bin"]=np.arange(nbins).repeat(times)  # agrega columna de etiquetas de bins
    return lo.sort_index().bin  # devuelve serie de etiquetas con el orden que tenía df

In [ ]:
Ifsoac(lorentz[0], transform="antipolar", tam_punto=10)
Ifsoac(same_amount_bins(lorentz), transform="antipolar", tam_punto=10)
Ifsoac(rossler[0], transform="antipolar", tam_punto=10)
Ifsoac(same_amount_bins(rossler), transform="antipolar", tam_punto=10)

In [ ]:
Ifsoac(same_amount_bins(rossler))
Ifsoac(same_amount_bins(logi))
Ifsoac(same_amount_bins(tienda))

# Descorrelacionando Lorentz y Rossler tomando un dato cada 10

Pedro Miramontes_Vidal, [03/02/22 13:56]
Okas.

La conjetura es entonces que todos los mapeos del intervalo de la familia logística tienen la misma representación con esta IFS. 

Ya entiendo lo de Róssler y Lorenz; dos datos sucesivos están muuuuy correlacionados pues son casi el mismo pues vienen de la discretización de una ecuación diferencial con un paso muy pequeño. Para evitar esto, hay que extraer de la serie de tiempo un dato cada 10 lugares y así formal una nueva serie de tiempo y analizarla con tus herramientas.

In [ ]:
srossler = same_amount_bins(rossler)
slorentz = same_amount_bins(lorentz)

In [ ]:
Ifsoac(slorentz[::10], tam_punto=5)
Ifsoac(srossler[::10], tam_punto=5)
Ifsoac(slorentz[::10], tam_punto=10, transform="antipolar", scale=0)
Ifsoac(srossler[::10], tam_punto=10, transform="antipolar", scale=0)

In [ ]:
slogi = same_amount_bins(logi)
stienda = same_amount_bins(tienda)

In [ ]:
Ifsoac(slogi, tam_punto=5)
Ifsoac(slogi[::2], tam_punto=5)
Ifsoac(slogi[::3], tam_punto=5)
Ifsoac(slogi[::4], tam_punto=5)



#Ifsoac(stienda[np.arange(0,len(stienda), cada)], tam_punto=5)

# Mapeo unimodal cóncavo

$$f(x) = \lambda(1-|1-2x|^{\nu})$$

In [ ]:
nu_inic = 2
nu_final = .5
_iteraciones = 12
for i in np.linspace(nu_inic,nu_final,_iteraciones, endpoint=True):
    _lambda = .999
    concavo = iterar(lambda x: _lambda*(1-np.power(np.abs(1-2*x),i)), N0=0, x0=.7)
    Ifsoac(same_amount_bins(concavo))

# Más valores para Lorentz

In [ ]:
def lorentz_array(N=500000):
    x0=1
    y0=1
    z0=3
    h=0.01
    sigma=10.0
    beta=8/3.0
    ro=28.0
    ll=[]
    for i in range(N):
        x1=h*(sigma*(y0-x0))+x0
        y1=h*(x0*(ro-z0)-y0)+y0
        z1=h*(x0*y0-beta*z0)+z0
        x0=x1
        y0=y1
        z0=z1
        ll.append(x1) #,y1,z1)
    return np.array(ll)


In [ ]:
la = lorentz_array(1000000)
for i in range(5,31,5):
    Ifsoac(la[::i])